In [26]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.applications import ResNet50
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax

In [2]:
train_dir = "/kaggle/input/ck-dataset"
num_classes = len(os.listdir(train_dir))

In [32]:
def grayscale_to_rgb(image, output_path):
    img = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.imwrite(output_path, img)

In [33]:
for folder in os.listdir(train_dir):
    folder_path = os.path.join(train_dir, folder)
    for image in os.listdir(folder_path):
        img_path = os.path.join(folder_path, image)
        output_dir = "/kaggle/working/"+folder_path.split('/')[-1]
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, image)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        grayscale_to_rgb(img, output_path)

In [34]:
img = cv2.imread('/kaggle/input/ck-dataset/anger/S010_004_00000017.png')
print(img.shape)
img_size = img.shape[0]

(48, 48, 3)


In [4]:
train_datagen = ImageDataGenerator(width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  rescale = 1./255,
                                  zoom_range = 0.2,
                                  validation_split=0.25)

In [5]:
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                  zoom_range = 0.2,
                                  validation_split=0.25)

In [32]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (224, 224),
                                                    batch_size = 64,
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training")
validation_generator = validation_datagen.flow_from_directory( directory = train_dir,
                                                              target_size = (224, 224,3),
                                                              batch_size = 64,
                                                              color_mode = "grayscale",
                                                              class_mode = "categorical",
                                                              subset = "validation")

Found 739 images belonging to 7 classes.
Found 242 images belonging to 7 classes.


In [21]:
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_base.trainable = False

In [ ]:
resnet_base.summary()

In [24]:
x = Flatten() (resnet_base.output)
prediction = Dense(len(os.listdir(train_dir)), activation = 'softmax')(x)

In [27]:
model = Model(inputs = resnet_base.input, outputs = prediction)

In [28]:
model.compile (
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_6[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [30]:
from tensorflow.keras.callbacks import *
checkpoint = ModelCheckpoint('emotion_detection.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             verbose=1,
                             mode='auto',
                             save_weights_only=False,
                             restore_best_weights=True,
                             period=1)

In [33]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 64,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 64,
    epochs=2,
    callbacks=checkpoint
)

ValueError: could not broadcast input array from shape (224,224,1) into shape (224,224,3,1)